In [4]:
import requests
import os

In [15]:
#os.environ['API_USER'] = 'xxx'
#os.environ['API_PASSWORD'] = 'xxx'

username = os.environ.get('API_USER')
password = os.environ.get('API_PASSWORD')

login_response = requests.post('https://m2m.cr.usgs.gov/api/api/json/stable/login', json = {"username": username, "password": password})

In [21]:
auth_code = login_response.json()['data']

In [23]:

payload = { 
    "datasetName": "landsat_ard_tile_c2" ,
    "sceneFilter": {
        "acquisitionFilter": {
            "start": "2018-01-01",
            "end": "2023-01-01"
        },
        "cloudCoverFilter": {
            "max": 3,
            "min": 0,
            "includeUnknown": False
        }#,
        #"spatialFilter": {
        #    "filterType": "mbr",
        #    "lowerLeft": {
        #            "latitude": 40,
        #            "longitude": -120
        #    },
        #    "upperRight": {
        #            "latitude": 50,
        #            "longitude": -100
        #    }
        #}
    }
}


response = requests.post('https://m2m.cr.usgs.gov/api/api/json/stable/scene-search', json = payload, headers={"X-Auth-Token": auth_code})
print(response.json())

{'requestId': 1116142427, 'version': 'stable', 'sessionId': 191047188, 'data': {'results': [{'browse': [{'id': '60fac014d0e80a74', 'browseRotationEnabled': None, 'browseName': 'Reflective Browse', 'browsePath': 'https://landsatlook.usgs.gov/gen-browse?size=rrb&type=t_refl&product_id=LC08_CU_006005_20230101_20230114_02', 'overlayPath': 'https://landsatlook.usgs.gov/dynamic-tiler/scenes/LC08_CU_006005_20230101_20230114_02/tiles/{z}/{x}/{y}.png?layer=natural_color', 'overlayType': 'ls_chs', 'thumbnailPath': 'https://landsatlook.usgs.gov/gen-browse?size=thumb&type=t_refl&product_id=LC08_CU_006005_20230101_20230114_02'}], 'cloudCover': 1, 'entityId': 'LC08_CU_006005_20230101_20230114_02', 'displayId': 'LC08_CU_006005_20230101_20230114_02', 'orderingId': None, 'metadata': [], 'hasCustomizedMetadata': None, 'options': {'bulk': True, 'download': True, 'order': False, 'secondary': False}, 'selected': {'bulk': False, 'compare': False, 'order': False}, 'spatialBounds': {'type': 'Polygon', 'coordi